In [1]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('AAPL_daily_data.csv')

# Rename "Unnamed: 0" to "Date" and parse it as a datetime column
df.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# Sort data by date (oldest to newest)
df = df.sort_values(by="Date").reset_index(drop=True)

In [2]:

import pandas as pd

def calculate_true_range(df):
    """Calculate True Range for ATR."""
    df['High-Low'] = df['high'] - df['low']
    df['High-Close'] = abs(df['high'] - df['close'].shift(1))
    df['Low-Close'] = abs(df['low'] - df['close'].shift(1))
    df['True Range'] = df[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
    return df
# Volatility signal
def calculate_atr(df, period=14):
    """Calculate Average True Range (ATR) using Wilder's smoothing."""
    df = calculate_true_range(df)
    df['ATR'] = df['True Range'].ewm(alpha=1/period, adjust=False, min_periods=1).mean()
    return df
# Stock trend discovery
def calculate_adx(df, period=14):
    """Calculate Average Directional Index (ADX)."""
    df = calculate_atr(df, period)  # Ensure ATR uses Wilder's smoothing

    # Calculate +DM and -DM
    df['+DM'] = df['high'] - df['high'].shift(1)  # Up move
    df['-DM'] = df['low'].shift(1) - df['low']    # Down move (previous_low - current_low)

    # Set negative DM values to 0
    df['+DM'] = df['+DM'].clip(lower=0)
    df['-DM'] = df['-DM'].clip(lower=0)

    # Zero out the smaller DM
    mask = df['+DM'] > df['-DM']
    df.loc[~mask, '+DM'] = 0
    df.loc[mask, '-DM'] = 0

    # Smooth +DM and -DM using Wilder's method
    df['Smoothed+DM'] = df['+DM'].ewm(alpha=1/period, adjust=False).mean()
    df['Smoothed-DM'] = df['-DM'].ewm(alpha=1/period, adjust=False).mean()

    # Calculate +DI and -DI
    df['+DI'] = (df['Smoothed+DM'] / df['ATR']) * 100
    df['-DI'] = (df['Smoothed-DM'] / df['ATR']) * 100

    # Calculate DX and ADX (smoothed DX)
    df['DX'] = (abs(df['+DI'] - df['-DI']) / (df['+DI'] + df['-DI'])) * 100
    df['ADX'] = df['DX'].ewm(alpha=1/period, adjust=False).mean()

    return df
# Volume weights
def calculate_obv(df):
    """Calculate On-Balance Volume (OBV)."""
    df['OBV'] = (df['volume'] * ((df['close'].diff() > 0) * 2 - 1)).cumsum()
    return df
# Buy&Sell signals
def calculate_rsi(df, period=14):
    """Calculate Relative Strength Index (RSI)."""
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period, min_periods=1).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period, min_periods=1).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    df['RSI'] = df['RSI'].fillna(50)  # Default initial RSI to 50
    return df
# Buy&Sell signals
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    """Calculate Moving Average Convergence Divergence (MACD)."""
    df['EMA_12'] = df['close'].ewm(span=short_window, adjust=False, min_periods=1).mean()
    df['EMA_26'] = df['close'].ewm(span=long_window, adjust=False, min_periods=1).mean()
    df['MACD'] = df['EMA_12'] - df['EMA_26']
    df['Signal_Line'] = df['MACD'].ewm(span=signal_window, adjust=False, min_periods=1).mean()
    df['MACD_Histogram'] = df['MACD'] - df['Signal_Line']
    return df

def calculate_technical_indicators(df):
    """Compute technical indicators including ADX, ATR, OBV, RSI, and MACD."""
    df = df.copy()
    df = calculate_atr(df, period=14)
    df = calculate_adx(df, period=14)
    df = calculate_obv(df)
    df = calculate_rsi(df, period=14)
    df = calculate_macd(df, short_window=12, long_window=26, signal_window=9)
    return df


In [3]:
# Apply the technical indicators to the dataset
df = calculate_technical_indicators(df)

# Generate Summary Table
summary_df = df[["Date", "close", "ATR", "ADX", "OBV", "RSI", "MACD", "Signal_Line", "MACD_Histogram"]].tail(10)

# Save results to CSV
df.to_csv("AAPL_technical_analysis.csv", index=False)
summary_df.to_csv("AAPL_summary_table.csv", index=False)

# Display the summary table in a readable format
print(summary_df.to_string(index=False))

,Date,open,high,low,close,volume,High-Low,High-Close,Low-Close,True Range,...,-DI,DX,ADX,OBV,RSI,EMA_12,EMA_26,MACD,Signal_Line,MACD_Histogram
0,1999-11-01,80.00,80.69,77.3700,77.62,2487300,3.3200,NaN,NaN,3.3200,...,NaN,NaN,NaN,-2487300,50.000000,77.620000,77.620000,0.000000,0.000000,0.000000
1,1999-11-02,78.00,81.69,77.3100,80.25,3564600,4.3800,4.07,0.3100,4.3800,...,0.000000,100.000000,100.000000,1077300,100.000000,78.024615,77.814815,0.209801,0.041960,0.167840
2,1999-11-03,81.62,83.25,81.0000,81.50,2932700,2.2500,3.00,0.7500,3.0000,...,0.000000,100.000000,100.000000,4010000,100.000000,78.559290,78.087791,0.471498,0.127868,0.343631
3,1999-11-04,82.06,85.37,80.6200,83.62,3384700,4.7500,3.87,0.8800,4.7500,...,0.000000,100.000000,100.000000,7394700,100.000000,79.337861,78.497585,0.840276,0.270349,0.569927
4,1999-11-05,84.62,88.37,84.0000,88.31,3721500,4.3700,4.75,0.3800,4.7500,...,0.000000,100.000000,100.000000,11116200,100.000000,80.718190,79.224430,1.493760,0.515031,0.978728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6361,2025-02-14,241.25,245.55,240.9900,244.60,40896227,4.5600,4.02,0.5400,4.5600,...,17.202857,30.536939,18.418021,-2518912928,64.955357,235.511931,235.465421,0.046510,-1.669417,1.715927
6362,2025-02-18,244.15,245.18,241.8400,244.47,48822491,3.3400,0.58,2.7600,3.3400,...,16.506430,30.536939,19.283658,-2567735419,57.571324,236.890096,236.132427,0.757669,-1.184000,1.941669
6363,2025-02-19,244.66,246.01,243.1604,244.87,32204215,2.8496,1.54,1.3096,2.8496,...,15.914473,32.085139,20.198050,-2535531204,56.834532,238.117773,236.779655,1.338119,-0.679576,2.017695
6364,2025-02-20,244.94,246.78,244.2900,245.83,32316907,2.4900,1.91,0.5800,2.4900,...,15.394939,33.564488,21.152795,-2503214297,60.430380,239.304270,237.450051,1.854219,-0.172817,2.027036


      Date  close      ATR       ADX         OBV       RSI      MACD  Signal_Line  MACD_Histogram
2025-02-07 227.63 6.249929 18.951397 -2745500508 47.600572 -2.411660    -2.650009        0.238349
2025-02-10 227.65 6.045291 17.727087 -2712384863 56.014406 -2.595056    -2.639018        0.043962
2025-02-11 232.62 6.154913 17.162937 -2658666501 59.674224 -2.312703    -2.573755        0.261053
2025-02-12 236.87 6.163848 16.921487 -2613423209 63.340739 -1.726098    -2.404224        0.678126
2025-02-13 241.53 6.207137 17.485797 -2559809155 67.592419 -0.875100    -2.098399        1.223299
2025-02-14 244.60 6.089485 18.418021 -2518912928 64.955357  0.046510    -1.669417        1.715927
2025-02-18 244.47 5.893093 19.283658 -2567735419 57.571324  0.757669    -1.184000        1.941669
2025-02-19 244.87 5.675701 20.198050 -2535531204 56.834532  1.338119    -0.679576        2.017695
2025-02-20 245.83 5.448151 21.152795 -2503214297 60.430380  1.854219    -0.172817        2.027036
2025-02-21 245.55 5.